In [1]:
import numpy as np
import matplotlib.pyplot as plt
from database_methods import Table, to_show, head
import sqlite3
con = sqlite3.connect('database_results.db')
cursor = con.cursor()

In [2]:
dg = Table(connection=con, name='masters')
exp = Table(con, name = 'experiments')
meas = Table(con, name = 'measurements')

In [3]:
print(dg.columns)
print(exp.columns)
print(meas.columns)
print('-'*80)
# exp.show_all()

CATEGORY, PERIOD, AMPLITUDE, MASTER
DATE, MASTER_ID, DIP, FOLDER
EXPERIMENT_ID, NP, WAVELENGTH, FILE, AR_TH, AR_DIP, AR_NOISE, EXC_THS, RATE, INFO, COUNT, X, Y, CON, INP, INPPX, IBGPX, STD, stdX, stdY, stdCON, stdINP, stdINPPX, stdIBGPX, stdSTD, NOTES, AR_MIN, AR_COND, EXP_TYPE
--------------------------------------------------------------------------------


In [4]:
data = con.execute("""
SELECT NP, WAVELENGTH, DIP, DATE, COUNT, CON
FROM 'experiments' as EXP INNER JOIN 'measurements' as MEAS
   ON EXP.ID = MEAS.EXPERIMENT_ID;
   """)
for row in data:
    print(str(row).replace(', ', '\t').replace("'", "")[1:-1])

80	740	727	20-02-26	128	3.7494866284721313
80	732	727	20-02-26	206	4.424635611736389
80	734	727	20-02-26	169	5.380528060883221
80	736	727	20-02-26	163	2.907049071433627
80	726	727	20-02-26	113	3.6359160741949874
80	724	727	20-02-26	137	3.053237526603112
60	732	727	20-02-26	166	3.968417288931015
60	732	727	20-02-26	234	3.816722879969379
80	651	647	20-03-16	54	3.8884484367508425
80	647	647	20-03-16	55	1.9014506031657823
80	649	647	20-03-16	66	2.9057549932215254


In [3]:
# 
columns = 'MEAS.ID, WAVELENGTH, NP, DIP, FILE, DATE, RATE, CON, INP, MASTER, AR_MIN, AR_COND, EXP_TYPE,  NOTES'
data = con.execute("""
SELECT {}
FROM 'masters' as MAS 
INNER JOIN 'experiments' as EXP 
ON MAS.ID = EXP.MASTER_ID
INNER JOIN 'measurements' as MEAS
ON EXP.ID = MEAS.EXPERIMENT_ID
ORDER BY CON DESC
   """.format(columns))

head(columns)

for row in data:
    print(str(to_show(row)).replace(', ', '\t').replace("'", "")[1:-1])
    
	

MEAS.ID	WAVELEN	NP	DIP	FILE	DATE	RATE	CON	INP	MASTER	AR_MIN	AR_COND	EXP_TYP	 NOTES
1	740	80	727	raw_07_	20-02-2	59.20	3.75	15.91	Q4	6	2	-	-
2	732	80	727	raw_04_	20-02-2	85.33	4.42	46.00	Q4	10	2	-	-
3	734	80	727	raw_05_	20-02-2	70.29	5.38	36.50	Q4	10	2	-	-
4	736	80	727	raw_06_	20-02-2	57.34	2.91	21.79	Q4	10	2	-	-
6	726	80	727	raw_08_	20-02-2	70.91	3.64	20.36	Q4	10	2	-	-
7	724	80	727	raw_09_	20-02-2	89.81	3.05	20.65	Q4	10	2	-	-
8	732	60	727	raw_18_	20-02-2	97.04	3.97	31.03	Q4	10	2	-	-
9	732	60	727	raw_21_	20-02-2	96.97	3.82	30.71	Q4	10	2	sh	-
10	651	80	647	raw_05_	20-03-1	29.07	3.89	10.07	K4	6	2	-	-
11	647	80	647	raw_03_	20-03-1	15.56	1.90	3.11	K4	6	1	-	-
12	649	80	647	raw_04_	20-03-1	26.32	2.91	5.14	K4	6	2	-	-
13	653	80	647	raw_06_	20-03-1	31.72	4.65	8.89	K4	6	1	-	-


In [40]:
columns = 'NP, WAVELENGTH, X'
data = con.execute("""
SELECT {} FROM 'measurements';
   """.format(columns))

print(columns.replace(', ','\t'))
for row in data:
    print(str(to_show(row)).replace(', ', '\t').replace("'", "")[1:-1])

NP	WAVELENGTH	X
80	740	16.71
80	732	32.05
80	734	24.95
80	736	23.03
80	726	41.31
80	724	40.87
60	732	27.16
60	732	28.07


In [95]:
con.execute("""
UPDATE 'measurements' SET NOTES = '-'
""")

In [97]:
meas.commit()

In [9]:
con.close()